# Road Crash Fatality Data - ETL Process for Neo4j Graph Database

## Project Overview

This notebook transforms the **ARDD Road Crash Fatality dataset** into CSV files optimized for Neo4j import. We follow a systematic ETL approach to create a 6-node graph schema designed for road safety analysis.

### Dataset Information
- **Source**: Project2_Dataset_Corrected.csv
- **Records**: 10,490 fatality records
- **Columns**: 25 attributes
- **Grain**: One row per person killed in a crash
- **Key Identifiers**:
  - `ID` = Unique person/fatality identifier
  - `Crash ID` = Crash event identifier (can have multiple fatalities)


### Target Graph Schema
Our 6-node design follows graph database best practices:
1. **CRASH** - Central hub (crash events)
2. **PERSON** - Individual fatalities
3. **LOCATION** - Geographic context
4. **DATETIME** - Date/time context
5. **VEHICLE_INVOLVEMENT** - Vehicle characteristics
6. **SPECIAL_PERIOD** - Holiday classifications


## Step 1: Environment Setup and Data Loading

In [2]:
# Import required libraries
import pandas as pd
import numpy as np

# Upload the dataset file to Google Colab
from google.colab import files
print("Please upload your Project2_Dataset_Corrected.csv file:")
uploaded = files.upload()

Please upload your Project2_Dataset_Corrected.csv file:


Saving Project2_Dataset_Corrected.csv to Project2_Dataset_Corrected.csv


In [3]:
# Load the dataset and perform initial exploration
df = pd.read_csv('Project2_Dataset_Corrected.csv')

print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Display first few records to understand structure
print("\nDataset Preview:")
df.head(3)

Shape: 10,490 rows × 25 columns
Memory usage: 12.16 MB

Dataset Preview:


,ID,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Number Fatalities,Bus Involvement,...,Age,National Remoteness Areas,SA4 Name 2021,National LGA Name 2024,National Road Type,Christmas Period,Easter Period,Age Group,Day of week,Time of day
0,1,20241115,NSW,12,2024,Friday,4:00,Single,1,No,...,74,Inner Regional Australia,Riverina,Wagga Wagga,Arterial Road,Yes,No,65_to_74,Weekday,Night
1,2,20241125,NSW,12,2024,Friday,6:15,Single,1,No,...,19,Inner Regional Australia,Sydney - Baulkham Hills and Hawkesbury,Hawkesbury,Local Road,No,No,17_to_25,Weekday,Day
2,3,20246013,TAS,12,2024,Friday,9:43,Single,1,No,...,33,Inner Regional Australia,Launceston and North East,Northern Midlands,Local Road,Yes,No,26_to_39,Weekday,Day



## Step 2: Data Quality Assessment

### Check for Missing Values

In [4]:
# Check for missing values
missing_summary = df.isnull().sum()
missing_summary = missing_summary[missing_summary > 0].sort_values(ascending=False)
if len(missing_summary) > 0:
    print("Missing values found:")
    print(missing_summary)
else:
    print("No missing values detected")

No missing values detected


### Profile Categorical Fields

In [5]:
 #Profile key categorical fields
categorical_cols = ['State', 'Crash Type', 'Bus Involvement', 'Heavy Rigid Truck Involvement',
                   'Articulated Truck Involvement', 'Christmas Period', 'Easter Period',
                   'Road User', 'Gender', 'Age Group', 'Day of week', 'Time of day','Age']

for col in categorical_cols:
    if col in df.columns:
        unique_count = df[col].nunique()
        print(f"{col}: {unique_count} unique values")
        if unique_count <= 10:
            print(f"  Values: {list(df[col].unique())}")


State: 8 unique values
  Values: ['NSW', 'TAS', 'QLD', 'SA', 'VIC', 'ACT', 'NT', 'WA']
Crash Type: 2 unique values
  Values: ['Single', 'Multiple']
Bus Involvement: 2 unique values
  Values: ['No', 'Yes']
Heavy Rigid Truck Involvement: 2 unique values
  Values: ['No', 'Yes']
Articulated Truck Involvement: 2 unique values
  Values: ['No', 'Yes']
Christmas Period: 2 unique values
  Values: ['Yes', 'No']
Easter Period: 2 unique values
  Values: ['No', 'Yes']
Road User: 6 unique values
  Values: ['Driver', 'Passenger', 'Motorcycle rider', 'Pedestrian', 'Pedal cyclist', 'Motorcycle pillion passenger']
Gender: 2 unique values
  Values: ['Male', 'Female']
Age Group: 6 unique values
  Values: ['65_to_74', '17_to_25', '26_to_39', '40_to_64', '0_to_16', '75_or_older']
Day of week: 2 unique values
  Values: ['Weekday', 'Weekend']
Time of day: 2 unique values
  Values: ['Night', 'Day']
Age: 102 unique values


# Step 3: Create Node CSV Files

### 3.1 CRASH Nodes (Central Hub)

The crash node represents each unique crash event. Since multiple people can die in one crash, we need to deduplicate by Crash ID.

### Evidence Supporting Crash-Centric Model:

- Natural Aggregation Point: Crashes are the events that generate all other context

- Query Efficiency: Most analytical questions focus on crash characteristics

- Relationship Logic: Geography, time, vehicles, and periods all describe the crash event

- Industry Standard: Road safety analysis typically focuses on crash events

In [18]:
 # keeping one row per person with crash info
# This maintains the same row count across CRASH and PERSON nodes
crash_df = df[['ID', 'Crash ID', 'Number Fatalities', 'Speed Limit']].copy()
crash_df.columns = ['ID', 'crash_id', 'number_fatalities', 'speed_limit']



print(f"✓ CRASH nodes created: {len(crash_df):,} records (same as person records)")

# Preview the crash data
crash_df.tail()

✓ CRASH nodes created: 10,490 records (same as person records)


,ID,crash_id,number_fatalities,speed_limit
10485,10486,20144079,1,110
10486,10487,20145055,1,80
10487,10488,20144007,1,50
10488,10489,20145072,1,110
10489,10490,20145108,1,100



### 3.2 PERSON Nodes (Individual Fatalities)

Each person/fatality gets their own node since this is our unit of analysis.

In [19]:
# Create person nodes - one per fatality record
# Use the ID field as person identifier (each row = one person)
person_df = df[['ID', 'Road User', 'Gender','Age','Age Group']].copy()
person_df.columns = ['ID', 'road_user', 'gender','Age','age_group']

print(f"✓ PERSON nodes created: {len(person_df):,} fatality records")
print("\nDemographic breakdown:")
print(f"Age groups: {person_df['age_group'].value_counts().to_dict()}")
print(f"Road users: {person_df['road_user'].value_counts().to_dict()}")
print(f"Gender: {person_df['gender'].value_counts().to_dict()}")

# Preview the person data
person_df.tail()

✓ PERSON nodes created: 10,490 fatality records

Demographic breakdown:
Age groups: {'40_to_64': 3248, '26_to_39': 2313, '17_to_25': 2046, '75_or_older': 1332, '65_to_74': 1036, '0_to_16': 515}
Road users: {'Driver': 4954, 'Motorcycle rider': 1915, 'Passenger': 1834, 'Pedestrian': 1412, 'Pedal cyclist': 326, 'Motorcycle pillion passenger': 49}
Gender: {'Male': 7761, 'Female': 2729}


,ID,road_user,gender,Age,age_group
10485,10486,Driver,Male,36,26_to_39
10486,10487,Motorcycle rider,Male,34,26_to_39
10487,10488,Pedestrian,Male,82,75_or_older
10488,10489,Driver,Male,84,75_or_older
10489,10490,Motorcycle rider,Male,46,40_to_64


### 3.3 LOCATION Nodes (Geographic Context)

Location nodes represent unique geographic combinations for spatial analysis.


In [22]:
# First, let's check the actual column names
print("Checking actual column names for location fields:")
location_columns = [col for col in df.columns if any(keyword in col.lower()
                   for keyword in ['state', 'sa4', 'lga', 'remoteness', 'road type'])]
print("Found location columns:", location_columns)

# Use the correct column names (adjust these based on your actual data)
# Keep same row count as person records - don't deduplicate
location_df = df[['ID', 'State', 'SA4 Name 2021', 'National LGA Name 2024',
                 'National Remoteness Areas', 'National Road Type']].copy()
location_df.columns = ['ID', 'state', 'SA4', 'LGA', 'Remoteness', 'road_type']

print(f"✓ LOCATION data created: {len(location_df):,} records (same as person records)")
print(f"States represented: {location_df['state'].unique()}")
print(f"Unique locations: {location_df[['state', 'LGA']].drop_duplicates().shape[0]}")

# Preview location data
location_df.tail()

Checking actual column names for location fields:
Found location columns: ['State', 'National Remoteness Areas', 'SA4 Name 2021', 'National LGA Name 2024', 'National Road Type']
✓ LOCATION data created: 10,490 records (same as person records)
States represented: ['NSW' 'TAS' 'QLD' 'SA' 'VIC' 'ACT' 'NT' 'WA']
Unique locations: 509


,ID,state,SA4,LGA,Remoteness,road_type
10485,10486,SA,South Australia - South East,Wattle Range,Outer Regional Australia,National or State Highway
10486,10487,WA,Perth - Inner,Cambridge,Major Cities of Australia,National or State Highway
10487,10488,SA,Adelaide - North,Playford,Major Cities of Australia,Local Road
10488,10489,WA,Western Australia - Outback (South),Esperance,Remote Australia,National or State Highway
10489,10490,WA,Perth - South East,Belmont,Major Cities of Australia,National or State Highway


### 3.4 DATETIME Nodes (Date/Time Context)
Datetime nodes capture unique date/time combinations for time-based analysis.

In [21]:
from datetime import datetime
 # Keep same row count - don't deduplicate temporal data
datetime_df = df[['ID', 'Year', 'Month', 'Dayweek', 'Time', 'Day of week', 'Time of day']].copy()
datetime_df.columns = ['ID', 'year', 'month', 'dayweek', 'time', 'day_category', 'time_category']

print(f"✓ Datetime data created: {len(datetime_df):,} records (same as person records)")
print(f"Year range: {datetime_df['year'].min()} to {datetime_df['year'].max()}")
print(f"Unique time combinations: {datetime_df[['year', 'month', 'time']].drop_duplicates().shape[0]}")

# Preview temporal data
datetime_df.tail()

✓ Datetime data created: 10,490 records (same as person records)
Year range: 2014 to 2024
Unique time combinations: 8146


,ID,year,month,dayweek,time,day_category,time_category
10485,10486,2014,1,Thursday,14:40,Weekday,Day
10486,10487,2014,1,Tuesday,16:42,Weekday,Day
10487,10488,2014,1,Tuesday,20:00,Weekday,Night
10488,10489,2014,1,Tuesday,21:30,Weekday,Night
10489,10490,2014,1,Wednesday,11:47,Weekday,Day


### 3.5 VEHICLE_INVOLVEMENT Nodes (Vehicle Context)

Vehicle involvement nodes represent unique combinations of vehicle types in crashes.

In [23]:
# Keep same row count - vehicle involvement fields are already clean (Yes/No)
# No need to convert to boolean - keep original Yes/No values
vehicle_df = df[['ID', 'Crash Type', 'Bus Involvement', 'Heavy Rigid Truck Involvement', 'Articulated Truck Involvement']].copy()
vehicle_df.columns = ['ID', 'crash_type', 'bus', 'heavy_rigid_truck', 'articulated_truck']

print(f"✓ VEHICLE_INVOLVEMENT data created: {len(vehicle_df):,} records (same as person records)")
print(f"Unique vehicle profiles: {vehicle_df[['crash_type', 'bus', 'heavy_rigid_truck', 'articulated_truck']].drop_duplicates().shape[0]}")

# Show the clean Yes/No values
print("\nVehicle involvement distribution:")
print(f"Bus: {vehicle_df['bus'].value_counts().to_dict()}")
print(f"Heavy Rigid Truck: {vehicle_df['heavy_rigid_truck'].value_counts().to_dict()}")
print(f"Articulated Truck: {vehicle_df['articulated_truck'].value_counts().to_dict()}")

# Preview vehicle data
vehicle_df.tail()

✓ VEHICLE_INVOLVEMENT data created: 10,490 records (same as person records)
Unique vehicle profiles: 13

Vehicle involvement distribution:
Bus: {'No': 10300, 'Yes': 190}
Heavy Rigid Truck: {'No': 9774, 'Yes': 716}
Articulated Truck: {'No': 9593, 'Yes': 897}


,ID,crash_type,bus,heavy_rigid_truck,articulated_truck
10485,10486,Single,No,No,Yes
10486,10487,Single,No,No,No
10487,10488,Single,No,No,No
10488,10489,Single,No,No,No
10489,10490,Single,No,No,No


### 4.6 SPECIAL_PERIOD Nodes (Holiday Classifications)

Special period nodes represent holiday and regular time classifications.



In [24]:
# Extract just the columns you want
special_period_df = df[['ID', 'Christmas Period', 'Easter Period']].copy()

# Rename ID column if needed
special_period_df.columns = ['ID', 'Christmas Period', 'Easter Period']

print(f"✓ SPECIAL_PERIOD data created: {len(special_period_df):,} records")
print(f"Christmas Period distribution: {special_period_df['Christmas Period'].value_counts().to_dict()}")
print(f"Easter Period distribution: {special_period_df['Easter Period'].value_counts().to_dict()}")

special_period_df.tail()

✓ SPECIAL_PERIOD data created: 10,490 records
Christmas Period distribution: {'No': 10156, 'Yes': 334}
Easter Period distribution: {'No': 10336, 'Yes': 154}


,ID,Christmas Period,Easter Period
10485,10486,No,No
10486,10487,No,No
10487,10488,No,No
10488,10489,No,No
10489,10490,Yes,No


## DOWNLOADING CSV'S

In [25]:
# Export all 6 node types to CSV
crash_df.to_csv('crash_nodes.csv', index=False)
person_df.to_csv('person_nodes.csv', index=False)
location_df.to_csv('location_nodes.csv', index=False)
datetime_df.to_csv('datetime_nodes.csv', index=False)
vehicle_df.to_csv('vehicle_involvement_nodes.csv', index=False)
special_period_df.to_csv('special_period_nodes.csv', index=False)

In [26]:

from google.colab import files

print(" Downloading all CSV files for Neo4j import...")

# 1. CRASH nodes
files.download('crash_nodes.csv')
print("crash_nodes.csv downloaded")

# 2. PERSON nodes
files.download('person_nodes.csv')
print(" person_nodes.csv downloaded")

# 3. LOCATION nodes
files.download('location_nodes.csv')
print("location_nodes.csv downloaded")

# 4. DATETIME nodes
files.download('datetime_nodes.csv')
print("datetime_nodes.csv downloaded")

# 5. VEHICLE_INVOLVEMENT nodes
files.download('vehicle_involvement_nodes.csv')
print("vehicle_involvement_nodes.csv downloaded")

# 6. SPECIAL_PERIOD nodes
files.download('special_period_nodes.csv')
print(" special_period_nodes.csv downloaded")

print("\n All CSV files downloaded successfully!")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

crash_nodes.csv downloaded


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

vehicle_involvement_nodes.csv downloaded


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

 special_period_nodes.csv downloaded

 All CSV files downloaded successfully!
